# Exact Diagonalization Prerequisites
<b> Christina Lee

Category: Grad</b>

I was going to go at exact diagonalization of a 1D Heisenburg Spin Chain all in one post, but then I realized, even for a grad level post, I need to explain a lot of stuff.  So here I'm going to examine some of the tricks we will use when actually performing exact diagonalization.  

So what is a 1D Heisenburg Spin $\frac{1}{2}$ Chain? 

### Spin $\frac{1}{2} $
At each site, we can have a particle pointing up $| \uparrow \rangle$, down $|\downarrow \rangle$, or some super-position of the two.  

### Heisenburg
Our spin has three degrees of freedom.  Our Hamiltonian has the general form
\begin{equation}
{\cal H} = \sum_{\langle i,j \rangle} J_x S_i^x S_j^x + J_y S_i^y S_j^y + J_z S_i^z S_j^z
\end{equation}

In the limit of $J_z \rightarrow 0$, we regain the XY model, and when only one coupling constant remains we regain the Ising model.  

### 1D Chain
Spin only couples to two neighbors.

### One easy initialization cell.

`n`: The number of spins we will deal with.  
$2^n$: Dimension of our Hilbert Space.  Or more simply, the number of different eigenstates our system will have.

In [2]:
n=3
nstates=2^n

8

So Exact Diagonalization is often memory limited.  Thus we want to represent our states in the most compact format possible.  Luckily, if we are dealing with spin $\frac{1}{2}$, we can just use the `0`'s ($|\downarrow \rangle$) and `1`'s ($\uparrow \rangle$) of the machine.  If you are dealing with higher spin, you can use base 3, 4, etc...  Part of the reason I needed to create this seperate post was to examine working with binary data.

We will keep our states stored as Int, but Julia has operations we can perform to look at the binary format and change the bits.

In [3]:
# psi is an arrow of all our wavefunctions
psi=collect(0:(nstates-1))

# Lets look at each state both in binary and base 10
for p in psi
    println(bin(p,n+1),' ',p)
end

0000 0
0001 1
0010 2
0011 3
0100 4
0101 5
0110 6
0111 7


For handy reference, since we will be using them a lot in our calculations, lets store all the powers of two in an array. They are our placeholders.

In [7]:
powers2=2.^collect(0:(n))
for p in powers2
    println(bin(p,n+1),' ',p)
end

0001 1
0010 2
0100 4
1000 8


We could continue to look at the binary format of a number by calling `bin`, but that converts the number to an array of strings.  So instead we want to perform bitwise operations to determine what the binary format looks like in terms of numbers.

Julia's `&` is the bitwise operation and.  That means if I combine two numbers, it states the overlap between the two. 1 overlaps with 1; 2 overlaps with 2; 3 overlaps with 2 and 1.

We will use this to compute magnetization.

In [25]:
println("String \tp&powers2 \tArray form \tMagentization")
for p in psi
    println(bin(p,n+1),"\t",p&powers2,"\t"
    ,round(Int,(p&powers2)./powers2),"\t",sum(round(Int,(p&powers2)./powers2))) 
end

String 	p&powers2 	Array form 	Magentization
0000	[0,0,0,0]	[0,0,0,0]	0
0001	[1,0,0,0]	[1,0,0,0]	1
0010	[0,2,0,0]	[0,1,0,0]	1
0011	[1,2,0,0]	[1,1,0,0]	2
0100	[0,0,4,0]	[0,0,1,0]	1
0101	[1,0,4,0]	[1,0,1,0]	2
0110	[0,2,4,0]	[0,1,1,0]	2
0111	[1,2,4,0]	[1,1,1,0]	3


## Masks and Permuting Indices

The off diagonal elements of the Hamiltonian are the permutation of two neighboring elements in the array.  We can permute two indices by combining a mask with a bitwise XOR `$` .

In [21]:
testmask=3
testp=1
println(bin(testmask,2),' ',bin(testp,2),' ',bin(testp$testmask,2))

11 01 10


In [32]:
mask=[0;powers2]+[powers2;0]
mask=mask[2:end-1]

println("Mask base10 \tMask Binary \tSummed from")
for i in 1:length(mask)
    println(mask[i],"\t\t",bin(mask[i],n+1),"\t\t",bin(powers2[i],n+1),"\t",bin(powers2[i+1],n+1))
end

Mask base10 	Mask Binary 	Summed from
3		0011		0001	0010
6		0110		0010	0100
12		1100		0100	1000


In [18]:
for p in psi
    println(p,' ',bin(p,4),' ',p $ mask[1],' ',bin(p$mask[1],4)) 
end

0 0000 3 0011
1 0001 2 0010
2 0010 1 0001
3 0011 0 0000
4 0100 7 0111
5 0101 6 0110
6 0110 5 0101
7 0111 4 0100


Obviously in situations like `0`, that's not a permutation going from `0000` to `0011`.  We need to find a set of two neighbors that are different from each other first, before we then mask them to find the permuted value.